# 7. Chapter: Multi-Agents

In [1]:
import requests
from datetime import datetime, timedelta
from typing import Any
from pydantic import BaseModel, Field
from language_models.agent import (
    Agent,
    OutputType,
    PromptingStrategy,
    Workflow,
    WorkflowAgentStep,
    WorkflowFunctionStep,
    WorkflowTransformationStep,
)
from language_models.tools import Tool
from language_models.models.llm import OpenAILanguageModel
from language_models.proxy_client import ProxyClient
from language_models.settings import settings

In [2]:
proxy_client = ProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

In [3]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-4",
    max_tokens=500,
    temperature=0.2,
)

In [4]:
system_prompt = """You are an AI assistant designed to help users with a variety of tasks.

Extract all numbers from the user's input text."""

agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="{prompt}",
    prompt_variables=["prompt"],
    output_type=OutputType.ARRAY_INTEGER,
    prompting_strategy=PromptingStrategy.SINGLE_COMPLETION,
    verbose=True,
)

agent_step = WorkflowAgentStep(name="numbers", agent=agent)

class Function(BaseModel):
    numbers: list[int]

function_step = WorkflowFunctionStep(name="sort", inputs=Function, function=lambda numbers: sorted(numbers))

filter_step = WorkflowTransformationStep(name="numbers_greater_10", input_field="sort", transformation="filter", function=lambda number: number > 10)

In [5]:
class Prompt(BaseModel):
    prompt: str = Field(description="The user prompt")

workflow = Workflow(
    name="Data Extraction",
    description="Extracts numbers from a given text",
    steps=[agent_step, function_step, filter_step],
    inputs=Prompt,
    output="numbers_greater_10",
    verbose=True,
)

In [6]:
def current_date() -> datetime:
    return datetime.now()

current_date_tool = Tool(
    function=current_date,
    name="Current Date",
    description="Use this tool to access the current local date and time",
)

In [7]:
class USGeopoliticalSurveyEarthquakeAPI(BaseModel):
    """Class that implements the API interface."""

    start_time: str = Field(
        None,
        description=(
            "Limit to events on or after the specified start time. NOTE: All times use ISO8601 Date/Time format."
            + " Unless a timezone is specified, UTC is assumed."
        ),
    )
    end_time: str = Field(
        None,
        description=(
            "Limit to events on or before the specified end time. NOTE: All times use ISO8601 Date/Time format."
            + " Unless a timezone is specified, UTC is assumed."
        ),
    )
    limit: int = Field(
        20000,
        description=(
            "Limit the results to the specified number of events. NOTE: The service limits queries to 20000,"
            + " and any that exceed this limit will generate a HTTP response code 400 Bad Request."
        ),
    )
    min_depth: int = Field(
        -100,
        description="Limit to events with depth more than the specified minimum.",
    )
    max_depth: int = Field(
        1000,
        description="Limit to events with depth less than the specified maximum.",
    )
    min_magnitude: int = Field(
        None,
        description="Limit to events with a magnitude larger than the specified minimum.",
    )
    max_magnitude: int = Field(
        None,
        description="Limit to events with a magnitude smaller than the specified maximum.",
    )
    alert_level: str = Field(
        None,
        description=(
            "Limit to events with a specific PAGER alert level."
            + " The allowed values are: alert_level=green Limit to events with PAGER"
            + ' alert level "green". alert_level=yellow Limit to events with PAGER alert level "yellow".'
            + ' alert_level=orange Limit to events with PAGER alert level "orange".'
            + ' alert_level=red Limit to events with PAGER alert level "red".'
        ),
    )

def get_earthquakes(
    endpoint: str,
    start_time: datetime = (datetime.now() - timedelta(days=30)).date(),
    end_time: datetime = datetime.now().date(),
    limit: int = 20000,
    min_depth: int = -100,
    max_depth: int = 1000,
    min_magnitude: int | None = None,
    max_magnitude: int | None = None,
    alert_level: str | None = None,
) -> Any:
    params = {
        "format": "geojson",
        "starttime": start_time,
        "endtime": end_time,
        "limit": limit,
        "mindepth": min_depth,
        "maxdepth": max_depth,
        "minmagnitude": min_magnitude,
        "maxmagnitude": max_magnitude,
        "alertlevel": alert_level,
        "eventtype": "earthquake",
    }
    response = requests.get(
        f"https://earthquake.usgs.gov/fdsnws/event/1/{endpoint}",
        params=params,
        timeout=None,
    )
    return response.json()

def query_earthquakes(
    start_time: datetime = (datetime.now() - timedelta(days=30)).date(),
    end_time: datetime = datetime.now().date(),
    limit: int = 20000,
    min_depth: int = -100,
    max_depth: int = 1000,
    min_magnitude: int | None = None,
    max_magnitude: int | None = None,
    alert_level: str | None = None,
) -> Any:
    return get_earthquakes(
        endpoint="query",
        start_time=start_time,
        end_time=end_time,
        limit=limit,
        min_depth=min_depth,
        max_depth=max_depth,
        min_magnitude=min_magnitude,
        max_magnitude=max_magnitude,
        alert_level=alert_level,
    )

def count_earthquakes(
    start_time: datetime = (datetime.now() - timedelta(days=30)).date(),
    end_time: datetime = datetime.now().date(),
    limit: int = 20000,
    min_depth: int = -100,
    max_depth: int = 1000,
    min_magnitude: int | None = None,
    max_magnitude: int | None = None,
    alert_level: str | None = None,
) -> Any:
    return get_earthquakes(
        endpoint="count",
        start_time=start_time,
        end_time=end_time,
        limit=limit,
        min_depth=min_depth,
        max_depth=max_depth,
        min_magnitude=min_magnitude,
        max_magnitude=max_magnitude,
        alert_level=alert_level,
    )

query_earthquakes_tool = Tool(
    function=query_earthquakes,
    name="Query Earthquakes",
    description="Use this tool to search recent earthquakes",
    args_schema=USGeopoliticalSurveyEarthquakeAPI,
)

count_earthquakes_tool = Tool(
    function=count_earthquakes,
    name="Count Earthquakes",
    description="Use this tool to count and aggregate recent earthquakes",
    args_schema=USGeopoliticalSurveyEarthquakeAPI,
)

In [8]:
system_prompt = "You are an United States Geological Survey expert who can answer questions regarding earthquakes."

earthquake_agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="{question}",
    prompt_variables=["question"],
    output_type=OutputType.STRING,
    tools=[current_date_tool, count_earthquakes_tool, query_earthquakes_tool],
    prompting_strategy=PromptingStrategy.CHAIN_OF_THOUGHT,
)

### Tools ###

You have access to the following tools:
- Tool Name: Current Date, Tool Description: Use this tool to access the current local date and time, Tool Input: None

- Tool Name: Count Earthquakes, Tool Description: Use this tool to count and aggregate recent earthquakes, Tool Input: {'start_time': {'default': None, 'description': 'Limit to events on or after the specified start time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.', 'title': 'Start Time', 'type': 'string'}, 'end_time': {'default': None, 'description': 'Limit to events on or before the specified end time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.', 'title': 'End Time', 'type': 'string'}, 'limit': {'default': 20000, 'description': 'Limit the results to the specified number of events. NOTE: The service limits queries to 20000, and any that exceed this limit will generate a HTTP response code 400 Bad Request.', 'titl

In [9]:
class EarthquakeQuery(BaseModel):
    question: str = Field(description="The earthquake related user question")

earthquake_chain = Workflow(
    name="Earthquake Agent",
    description="Allows you to answer earthquake related questions",
    inputs=EarthquakeQuery,
    output="earthquake",
    steps=[WorkflowAgentStep(name="earthquake", agent=earthquake_agent)],
    verbose=True,
)

In [10]:
almighty_agent = Agent.create(
    llm=llm,
    system_prompt=system_prompt,
    prompt="{prompt}",
    prompt_variables=["prompt"],
    tools=[workflow.as_tool(), earthquake_chain.as_tool()],
    output_type=OutputType.STRING,
    prompting_strategy=PromptingStrategy.CHAIN_OF_THOUGHT,
    verbose=True,
)

### Tools ###

You have access to the following tools:
- Tool Name: Data Extraction, Tool Description: Extracts numbers from a given text, Tool Input: {'prompt': {'description': 'The user prompt', 'title': 'Prompt', 'type': 'string'}}

- Tool Name: Earthquake Agent, Tool Description: Allows you to answer earthquake related questions, Tool Input: {'question': {'description': 'The earthquake related user question', 'title': 'Question', 'type': 'string'}}

### Instructions ###

Your goal is to solve the problem you will be provided with

You should respond with:
```
Thought: <thought process on how to respond to the prompt>

Tool: <name of the tool to use>

Tool Input: <input of the tool to use>
```

Your <input of the tool to use> must be a JSON format with the keyword arguments of <name of the tool to use>

When you know the final answer to the user's query you should respond with:
```
Thought: <thought process on how to respond to the prompt>

Final Answer: <response to the prompt>
```

In [11]:
output = almighty_agent.invoke({"prompt": "How many earthquakes happened today?"})

Thought: The user is asking about the number of earthquakes that occurred today. I can use the Earthquake Agent tool to answer this question.
Tool: Earthquake Agent
Tool Input: {'question': 'How many earthquakes happened today?'}
Running Step: earthquake
Agent Input: {'question': 'How many earthquakes happened today?'}
Thought: To answer this question, I need to count the number of earthquakes that occurred today. I will use the "Current Date" tool to get today's date and then use the "Count Earthquakes" tool with the start_time set to the start of today and the end_time set to the end of today.
Tool: Current Date
Tool Input: {}
Tool Output: 2024-08-08 09:41:37.343613
Thought: The current date is August 8, 2024. I will use the "Count Earthquakes" tool with the start_time set to the start of today (2024-08-08T00:00:00) and the end_time set to the current time (2024-08-08T09:41:37).
Tool: Count Earthquakes
Tool Input: {'start_time': '2024-08-08T00:00:00', 'end_time': '2024-08-08T09:41:37

In [12]:
prompt = """Last weekend, six of us went on a 15-kilometer hike, starting at 7 AM.

By noon, we had covered 10 kilometers and reached Mount Elbert's 4,401-meter summit by 2 PM, with a temperature of 12°C.

We camped 5 kilometers away by 6 PM with 12 others and returned home by 5 PM the next day."""

output = almighty_agent.invoke({"prompt": prompt})

Thought: The user is sharing a story about their hike. There doesn't seem to be a question or problem to solve here. However, there are several numbers mentioned that could be relevant if a question is asked later. I can use the Data Extraction tool to identify these numbers and their context for future reference.
Tool: Data Extraction
Tool Input: {'prompt': "Last weekend, six of us went on a 15-kilometer hike, starting at 7 AM. By noon, we had covered 10 kilometers and reached Mount Elbert's 4,401-meter summit by 2 PM, with a temperature of 12°C. We camped 5 kilometers away by 6 PM with 12 others and returned home by 5 PM the next day."}
Running Step: numbers
Agent Input: {'prompt': "Last weekend, six of us went on a 15-kilometer hike, starting at 7 AM. By noon, we had covered 10 kilometers and reached Mount Elbert's 4,401-meter summit by 2 PM, with a temperature of 12°C. We camped 5 kilometers away by 6 PM with 12 others and returned home by 5 PM the next day."}
Final Answer: [6, 15,